In [31]:
import sys
sys.path.append(R'E:\Data science\Titanic dataset\notebooks')

from auto_imports import model_results_imports ,auto_imports

model_results_imports()
auto_imports()

import joblib  # to laod the models
predict_data = pd.read_csv('E:\Data science\Titanic dataset\data\Processed data\Data Modeling\predicting_data.csv')

In [32]:
# loading the models
logit = joblib.load('E:\Data science\Titanic dataset\Models\LogisticRegression.pkl')
RF_ = joblib.load('E:\Data science\Titanic dataset\Models\RandomForest.pkl')
svm_ = joblib.load('E:\Data science\Titanic dataset\Models\SupportVectorMachine.pkl')

## Manual Ensembling with Model-Specific Thresholds

This notebook demonstrates how to ensemble three different models (SVM, Logistic Regression, Random Forest) using specific probability thresholds for each. Each model's prediction is binarized using its own threshold, and the final ensemble prediction is determined by majority voting.

### Ensemble Approach

- **Step 1:** Obtain predicted probabilities for each model on the test data.
- **Step 2:** Apply a specific threshold to each model's probabilities to get binary predictions:
  - SVM: threshold = 0.44
  - Logistic Regression: threshold = 0.59
  - Random Forest: threshold = 0.6
- **Step 3:** Combine the binary predictions using majority voting (at least 2 out of 3 models must predict 1 for the ensemble to predict 1).

In [33]:
# getting probabilities 
svm_probs = svm_.predict_proba(predict_data.drop('PassengerId' , axis=1))[: , 1]
RF_probs = RF_.predict_proba(predict_data.drop('PassengerId' , axis=1))[: , 1]
logit_probs = logit.predict_proba(predict_data.drop('PassengerId' , axis=1))[: , 1]


# get predictions
svm_preds = (svm_probs >= 0.44).astype(int)
logit_preds = (logit_probs >= 0.59).astype(int)
RF_preds = (RF_probs >= 0.6).astype(int)

In [34]:
# Majority voting manually
ensemble_preds = ( (svm_preds + logit_preds + RF_preds) >= 2).astype(int)

In [35]:
ensemble_preds[:5]

array([0, 0, 0, 0, 1])

In [36]:
pd.DataFrame(
    {
        'PassengerId':predict_data['PassengerId'],
        'Survived': ensemble_preds
    }
).to_csv(r'E:\Data science\Titanic dataset\data\Processed data\final_predictions.csv' ,index=False )

### Notes

- This approach allows each model to contribute its prediction based on its optimal threshold, which can improve ensemble performance compared to using a single threshold for all models.
- The final predictions are saved to `final_predictions.csv` for submission or further analysis.